In [1]:
from src import utils
from copy import deepcopy
from sklearn.preprocessing import  OneHotEncoder

import numpy as np
import pandas as pd

In [2]:
deserialize_data = utils.deserialize_data

X_TRAIN_PATH = "data/interim/X_train.pkl"
X_train = deserialize_data(X_TRAIN_PATH)

Y_TRAIN_PATH = "data/interim/y_train.pkl"
y_train = deserialize_data(Y_TRAIN_PATH)['loan_status']

X_TEST_PATH = "data/interim/x_test.pkl"
X_test = deserialize_data(X_TEST_PATH)

Y_TEST_PATH = "data/interim/y_test.pkl"
y_test = deserialize_data(Y_TEST_PATH)['loan_status']

X_VALID_PATH = "data/interim/x_valid.pkl"
X_valid = deserialize_data(X_VALID_PATH)

Y_VALID_PATH = "data/interim/y_valid.pkl"
y_valid = deserialize_data(Y_VALID_PATH)['loan_status']

In [3]:
def drop_duplicate_data(X, y):
    """
    Fungsi untuk membuang data duplikat pada dataset dan targetnya.

    Parameters:
    X (pd.DataFrame): Data set (train, test, atau valid) yang ingin dibuang data duplikatnya.
    y (pd.Series): Data target yang sesuai dengan data pada parameter X.

    Returns:
    Tuple[pd.DataFrame, pd.Series]: Data X dan y setelah duplikat dibuang.
    """
    # Validasi tipe parameter X dan y
    if not isinstance(X, pd.DataFrame):
        raise TypeError("Parameter X harus bertipe pandas DataFrame.")
    if not isinstance(y, pd.Series):
        raise TypeError("Parameter y harus bertipe pandas Series.")
    if len(X) != len(y):
        raise ValueError("Parameter X dan y harus memiliki jumlah baris yang sama.")
    
    print("Fungsi drop_duplicate_data: parameter telah divalidasi.")
    
    # Copy data
    X = X.copy()
    y = y.copy()
    
    print(f"Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah {X.shape}.")
    
    # Seleksi data yang duplicate (True di duplicated())
    X_duplicate = X[X.duplicated(keep=False)]
    print(f"Fungsi drop_duplicate_data: shape dari data yang duplicate adalah {X_duplicate.shape}.")
    
    # Prediksi shape setelah drop duplicate
    rows_after_drop = X.shape[0] - X_duplicate.shape[0]
    X_clean = (rows_after_drop, X.shape[1])
    print(f"Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah {X_clean}.")
    
    # Drop duplicate inplace
    X.drop_duplicates(inplace=True)
    
    # Seleksi y dengan index baru X
    y = y.loc[X.index]
    
    print(f"Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah {X.shape}.")
    
    return X, y



def median_imputation(data, subset_data, fit):
    """
    Fungsi untuk imputasi data numerik dengan metode median.

    Parameters:
    data (pd.DataFrame): Dataset (train, test, atau valid) yang akan diimputasi.
    subset_data (list atau dict):
        - Jika fit=True, harus list berisi nama kolom yang ingin dihitung median-nya.
        - Jika fit=False, harus dict berisi nama kolom dan nilai median untuk imputasi.
    fit (bool): Jika True, fungsi akan menghitung median dari subset_data (list kolom).
                Jika False, fungsi akan melakukan imputasi berdasarkan dict subset_data.

    Returns:
    dict atau pd.DataFrame:
        - Jika fit=True, mengembalikan dict berisi median tiap kolom.
        - Jika fit=False, mengembalikan DataFrame hasil imputasi.
    """
    
    # Validasi tipe data
    if not isinstance(data, pd.DataFrame):
        raise RuntimeError("Fungsi median_imputation: parameter data haruslah bertipe DataFrame!")
    
    # Validasi parameter fit
    if fit == True:
        if not isinstance(subset_data, list):
            raise RuntimeError(
                "Fungsi median_imputation: untuk nilai parameter fit = True, "
                "subset_data harus bertipe list dan berisi daftar nama kolom yang ingin dicari nilai mediannya guna menjadi data imputasi pada kolom tersebut."
            )
    elif fit == False:
        if not isinstance(subset_data, dict):
            raise RuntimeError(
                "Fungsi median_imputation: untuk nilai parameter fit = False, "
                "subset_data harus bertipe dict dan berisi key yang merupakan nama kolom beserta value yang merupakan nilai median dari kolom tersebut."
            )
    else:
        raise RuntimeError(
            "Fungsi median_imputation: parameter fit haruslah bertipe boolean, bernilai True atau False."
        )
    
    print("Fungsi median_imputation: parameter telah divalidasi.")
    
    # Copy data untuk menghindari perubahan data asli
    data = data.copy()
    subset_data = deepcopy(subset_data)
    
    if fit == True:
        imputation_data = {}
        for subset in subset_data:
            median_val = data[subset].median()
            imputation_data[subset] = median_val
        print(f"Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {imputation_data}.")
        return imputation_data
    
    elif fit == False:
        print("Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:")
        print(data.isna().sum())
        print()
        
        # Imputasi berdasarkan nilai median yang diberikan
        data.fillna(value=subset_data, inplace=True)
        
        print("Fungsi median_imputation: informasi count na setelah dilakukan imputasi:")
        print(data.isna().sum())
        print()
        
        return data
    

serialize_data = utils.serialize_data

def create_onehot_encoder(categories, path):
    # Validasi parameter categories
    if not isinstance(categories, list):
        raise RuntimeError(
            "Fungsi create_onehot_encoder: parameter categories haruslah bertipe list, berisi kategori yang akan dibuat encodernya."
        )
    # Validasi parameter path
    if not isinstance(path, str):
        raise RuntimeError(
            "Fungsi create_onehot_encoder: parameter path haruslah bertipe string, berisi lokasi pada disk komputer dimana encoder akan disimpan."
        )
    
    # Buat instance OneHotEncoder
    ohe = OneHotEncoder(sparse_output=False)
    
    # Ubah tipe data categories dari list 1 dimensi menjadi numpy array 2 dimensi
    categories_reshaped = np.array(categories).reshape(-1, 1)
    
    # Fitting encoder
    ohe.fit(categories_reshaped)
    
    # Simpan encoder menggunakan fungsi serialize_data()
    serialize_data(ohe, path)
    
    # Print kategori yang telah dipelajari
    learned_categories = ohe.categories_[0].tolist()
    print(f"Kategori yang telah dipelajari adalah {learned_categories}")
    
    # Kembalikan encoder
    return ohe


import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def ohe_transform(dataset, subset, prefix, ohe):
    """
    Fungsi untuk melakukan encoding data kategorik
    
    Parameters:
    dataset (DataFrame): Set data yang ingin dilakukan pengkodean
    subset (str): Nama kolom yang terdapat pada data di parameter dataset
    prefix (str): Nama awalan yang akan disematkan pada kolom hasil pengkodean
    ohe (OneHotEncoder): Encoder yang sebelumnya telah dilatih oleh data kategorik khusus
    
    Returns:
    DataFrame: Dataset yang telah dilakukan pengkodean
    """
    
    # Percabangan pertama - Validasi parameter dataset
    if not isinstance(dataset, pd.DataFrame):
        raise RuntimeError("Fungsi ohe_transform: parameter dataset harus bertipe DataFrame!")
    
    # Percabangan kedua - Validasi parameter ohe
    if not isinstance(ohe, OneHotEncoder):
        raise RuntimeError("Fungsi ohe_transform: parameter ohe harus bertipe OneHotEncoder!")
    
    # Percabangan ketiga - Validasi parameter prefix
    if not isinstance(prefix, str):
        raise RuntimeError("Fungsi ohe_transform: parameter prefix harus bertipe str!")
    
    # Percabangan keempat - Validasi parameter subset
    if not isinstance(subset, str):
        raise RuntimeError("Fungsi ohe_transform: parameter subset harus bertipe str!")
    
    # Percabangan kelima - Pengecekan data pada parameter subset
    try:
        column_list = list(dataset.columns)
        column_list.index(subset)
    except ValueError:
        raise RuntimeError("Fungsi ohe_transform: parameter subset string namun data tidak ditemukan dalam daftar kolom yang terdapat pada parameter dataset.")
    
    print("Fungsi ohe_transform: parameter telah divalidasi.")
    
    # Buat duplikat dari data pada parameter dataset
    dataset = dataset.copy()
    
    # Print pesan yang menampilkan daftar nama kolom sebelum dilakukan pengkodean
    print(f"Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah {list(dataset.columns)}.")
    
    # Buat satu variabel bernama col_names untuk menyimpan nama kolom yang telah dikodekan
    col_names = [f"{prefix}_{col_name}" for col_name in ohe.categories_[0].tolist()]
    
    # Proses pengkodean:
    # Buat satu variabel bernama encoded
    transformed_data = ohe.transform(dataset[[subset]])
    
    # Cek apakah hasil transform adalah sparse matrix atau array
    if hasattr(transformed_data, 'toarray'):
        # Jika sparse matrix, convert ke array
        array_data = transformed_data.toarray()
    else:
        # Jika sudah array, gunakan langsung
        array_data = transformed_data
    
    encoded = pd.DataFrame(
        data=array_data,
        columns=col_names,
        index=dataset.index
    )
    
    # Proses penyatuan hasil pengkodean dengan data sebelum pengkodean
    dataset = pd.concat([dataset, encoded], axis=1)
    
    # Proses penghapusan kolom yang tidak diperlukan
    dataset.drop(columns=[subset], inplace=True)
    
    # Print pesan yang menandakan bahwa proses pengkodean telah berhasil
    print(f"Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah {list(dataset.columns)}.")
    
    # Kembalikan dataset dari fungsi
    return dataset

In [4]:
X_train, y_train = drop_duplicate_data(X_train, y_train)

Fungsi drop_duplicate_data: parameter telah divalidasi.
Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah (26064, 11).
Fungsi drop_duplicate_data: shape dari data yang duplicate adalah (192, 11).
Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah (25872, 11).
Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah (25968, 11).


In [5]:
# Contoh subset_data: list nama kolom numerik yang ingin diimputasi
subset_data = ['person_emp_length', 'loan_int_rate']  # sesuaikan dengan kolom di datasetmu

# Fit dan hitung median dari X_train pada kolom subset_data
subset_data = median_imputation(X_train, subset_data, True)  # outputnya dict kolom: median

# Imputasi X_train dengan median hasil fit tadi
X_train = median_imputation(X_train, subset_data, False)

# Imputasi X_test dengan median hasil fit tadi
X_test = median_imputation(X_test, subset_data, False)

# Imputasi X_valid dengan median hasil fit tadi
X_valid = median_imputation(X_valid, subset_data, False)


Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {'person_emp_length': np.float64(4.0), 'loan_int_rate': np.float64(10.99)}.
Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              734
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 2491
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_a

In [6]:
X_train.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length'],
      dtype='object')

In [7]:
person_home_ownership = list(X_train['person_home_ownership'].unique())
loan_intent = list(X_train['loan_intent'].unique())
loan_grade = list(X_train['loan_grade'].unique())
cb_person_default_on_file = list(X_train['cb_person_default_on_file'].unique())

# Panggil fungsi create_onehot_encoder() untuk person_home_ownership
ohe_home_ownership = create_onehot_encoder(
    categories=person_home_ownership,
    path='models/ohe_home_ownership.pkl'
)

# Panggil fungsi create_onehot_encoder() untuk loan_intent
ohe_loan_intent = create_onehot_encoder(
    categories=loan_intent,
    path='models/ohe_loan_intent.pkl'
)

# Panggil fungsi create_onehot_encoder() untuk loan_grade
ohe_loan_grade = create_onehot_encoder(
    categories=loan_grade,
    path='models/ohe_loan_grade.pkl'
)

# Panggil fungsi create_onehot_encoder() untuk cb_person_default_on_file
ohe_default_on_file = create_onehot_encoder(
    categories=cb_person_default_on_file,
    path='models/ohe_default_on_file.pkl'
)


Kategori yang telah dipelajari adalah ['MORTGAGE', 'OTHER', 'OWN', 'RENT']
Kategori yang telah dipelajari adalah ['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE']
Kategori yang telah dipelajari adalah ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Kategori yang telah dipelajari adalah ['N', 'Y']


In [8]:
# 1. Jalankan fungsi untuk melakukan encoding data kategorik

# Untuk X_train
print("=== Encoding X_train ===")

# 1. Untuk kolom person_home_ownership
X_train = ohe_transform(
    dataset=X_train,
    subset="person_home_ownership",
    prefix="home_ownership",
    ohe=ohe_home_ownership
)

# 2. Untuk kolom loan_intent
X_train = ohe_transform(
    dataset=X_train,
    subset="loan_intent",
    prefix="loan_intent",
    ohe=ohe_loan_intent
)

# 3. Untuk kolom loan_grade
X_train = ohe_transform(
    dataset=X_train,
    subset="loan_grade",
    prefix="loan_grade",
    ohe=ohe_loan_grade
)

# 4. Untuk kolom cb_person_default_on_file
X_train = ohe_transform(
    dataset=X_train,
    subset="cb_person_default_on_file",
    prefix="default_onfile",
    ohe=ohe_default_on_file
)

print("\n=== Encoding X_test ===")

# Untuk X_test
# 1. Untuk kolom person_home_ownership
X_test = ohe_transform(
    dataset=X_test,
    subset="person_home_ownership",
    prefix="home_ownership",
    ohe=ohe_home_ownership
)

# 2. Untuk kolom loan_intent
X_test = ohe_transform(
    dataset=X_test,
    subset="loan_intent",
    prefix="loan_intent",
    ohe=ohe_loan_intent
)

# 3. Untuk kolom loan_grade
X_test = ohe_transform(
    dataset=X_test,
    subset="loan_grade",
    prefix="loan_grade",
    ohe=ohe_loan_grade
)

# 4. Untuk kolom cb_person_default_on_file
X_test = ohe_transform(
    dataset=X_test,
    subset="cb_person_default_on_file",
    prefix="default_onfile",
    ohe=ohe_default_on_file
)

print("\n=== Encoding X_valid ===")

# Untuk X_valid
# 1. Untuk kolom person_home_ownership
X_valid = ohe_transform(
    dataset=X_valid,
    subset="person_home_ownership",
    prefix="home_ownership",
    ohe=ohe_home_ownership
)

# 2. Untuk kolom loan_intent
X_valid = ohe_transform(
    dataset=X_valid,
    subset="loan_intent",
    prefix="loan_intent",
    ohe=ohe_loan_intent
)

# 3. Untuk kolom loan_grade
X_valid = ohe_transform(
    dataset=X_valid,
    subset="loan_grade",
    prefix="loan_grade",
    ohe=ohe_loan_grade
)

# 4. Untuk kolom cb_person_default_on_file
X_valid = ohe_transform(
    dataset=X_valid,
    subset="cb_person_default_on_file",
    prefix="default_onfile",
    ohe=ohe_default_on_file
)

print("\n=== Encoding completed for all datasets ===")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_valid shape: {X_valid.shape}")

=== Encoding X_train ===
Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length'].
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT'].
Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', '

/home/fullkersa/BTG_MLPROCESS/ENV_JANCOK/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
/home/fullkersa/BTG_MLPROCESS/ENV_JANCOK/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
/home/fullkersa/BTG_MLPROCESS/ENV_JANCOK/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
/home/fullkersa/BTG_MLPROCESS/ENV_JANCOK/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
/home/fullkersa/BTG_MLPROCESS/ENV_JANCOK/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without f

In [9]:
# Serialize hasil encoding untuk X_train, X_test, dan X_valid
serialize_data(X_train, "data/processed/X_train_prep.pkl")
serialize_data(X_test, "data/processed/X_test_prep.pkl")
serialize_data(X_valid, "data/processed/X_valid_prep.pkl")
serialize_data(y_train,"data/processed/y_train_prep.pkl")
